## Creating Our Facts Table

In [0]:
df_fact = spark.read.table("flights_project.silver.silver_bookings")

In [0]:
df_fact.display()

In [0]:
df_aiport_dim = spark.read.table("flights_project.gold.dimairports")
df_passenger_dim = spark.read.table("flights_project.gold.dimpassengers")
df_flight_dim = spark.read.table("flights_project.gold.dimflights")


In [0]:
df_aiport_dim.display()
df_passenger_dim.display()
df_flight_dim.display()

In [0]:

df_facts = spark.sql("""
SELECT f.booking_id, f.booking_date, f.booking_amount, f.update_date, l.DimFlightsKey, p.DimPassengersKey, a.DimAirportsKey
FROM flights_project.silver.silver_bookings f
LEFT JOIN flights_project.gold.dimflights l ON f.flight_id = l.flight_id 
LEFT JOIN flights_project.gold.dimairports a ON f.airport_id = a.airport_id
LEFT JOIN flights_project.gold.dimpassengers p ON f.passenger_id = p.passenger_id 
""")

In [0]:
df_facts.display()

## UPSERT ON The Fact Table

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("flights_project.gold.bookings_facts"):
  dlt_obj = DeltaTable.forPath(spark, "abfss://gold@dbdatalakejess.dfs.core.windows.net/bookings_facts")
  dlt_obj.alias("trg").merge(df_facts.alias("src"), "trg.DimFlightsKey = src.DimFlightsKey AND trg.DimPassengersKey = src.DimPassengersKey AND trg.DimAirportsKey = src.DimAirportsKey")\
   .whenMatchedUpdateAll( condition = "src.update_date > trg.update_date")\
   .whenNotMatchedInsertAll()\
   .execute()

else:
  df_facts.write.format("delta").mode("overwrite").option("path", "abfss://gold@dbdatalakejess.dfs.core.windows.net/bookings_facts").saveAsTable("flights_project.gold.bookings_facts")


In [0]:
from pyspark.sql.functions import *

In [0]:
df = spark.sql("""

SELECT * FROM flights_project.gold.dimpassengers
""").groupBy("DimPassengersKey").count().filter(col('count') > 1)
df.display()